## 事前準備

4-0. ファイルが読み込めるかどうかの確認

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
# CSVファイルの読み込み
df = pd.read_csv('/content/drive/MyDrive/shuffled_medical_texts_labels.csv')
df

以降のセルはWS中に実施します

# WS4回目(自然言語処理2回目)

本演習内容はボリュームが多いため、notebookを実行しながら説明致します

# Transformerのtokenizerを使って学習を行う

(正確にはファインチューニング)

4-1.transformersのライブラリのインストール

In [ ]:
!pip install transformers==4.37.0

# トークナイザーの実践例

4-2.トークナイザーの指定と特定の事前学習済みモデルの重みの読み込み

In [ ]:
from transformers import BertTokenizer

# トークナイザーとそのトークナイザーで学習したチェックポイントの重みの読み込み
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

4-3.トークン化(tokenize)

In [ ]:
tokenized_en = tokenizer.tokenize("I have a pen")
tokenized_en

In [ ]:
tokenized_en2 = tokenizer.tokenize("The AI technology is evolving unpredictably.")
tokenized_en2

In [ ]:
tokenized_ja = tokenizer.tokenize("僕はペンを持っている。")
tokenized_ja

4-4.エンコーディング(encode)

In [ ]:
#引数は文字列
encoded_en = tokenizer.encode("I have a pen")
print(encoded_en)
type(encoded_en)

4-5. 日本語BERTでのTokenizer

In [ ]:
#fugashiは形態素解析ツール unidic_liteは辞書
!pip install fugashi
!pip install unidic_lite
from transformers import BertJapaneseTokenizer
tokenizer2 = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-v3')


In [ ]:
# トークン化
tokenized_ja2 = tokenizer2.tokenize("僕はペンを持っている。")
tokenized_ja2

4-6. 日本語版Bertでのエンコード

In [ ]:
# エンコード
encoded_ja2 = tokenizer2.encode("僕はペンを持っている。")
print(encoded_ja2)

# 複数の行をencodeさせるにはbatch_encode_plusを使用する.  


4-7.(batch_encode_plus).  
token_type_idsは文ごとにIDを振りたい時に用いる(デフォルトは0)  
attention_maskはbertモデルに単語(トークン)と認識させるかどうか(トークンであれば1)

In [ ]:
text_temp = ["僕はペンを持っている。","彼らは鉛筆を探しているが、見つからない。"]
temp2 = tokenizer2.batch_encode_plus(text_temp)
temp2

4-8. temp2を見やすく

In [ ]:
for i in range(2):
  print(f"{i+1}文目")
  print(temp2['input_ids'][i])
  print(temp2['attention_mask'][i])
  print(f"トークン長:{len(temp2['input_ids'][i])}")


↑.  
順に処理されて'input_ids'、'token_type_ids'、'attention_mask'に2つずつデータが格納されている.  
token_type_idsは全て0.  
attention_maskは全て1(=全てトークン).   
トークンの長さは文の長い2つ目の方が多くなっている

**全ての文のサイズを同じにする**.  
実際には複数の文を全て同じサイズにする.     
画像処理で行なったMLPやCNNも全て同じサイズにしていた.  
するとバッチサイズごとに行列演算などが可能になる.   

4-9.オプション(引数)を追加する.  
     

max_length：不足部分を補うトークンの最大長.  
padding：'max_length'にするとmax_length(下の場合14)のトークン長に合わせて不足部分を0で埋める.   
truncation：余剰部分もmax_lengthの長さを揃える

In [ ]:
temp3 = tokenizer2.batch_encode_plus(text_temp,
                                       max_length=14,
                                       padding='max_length',
                                       truncation=True,
                                       )
print(temp3)
for i in range(2):
  print(f"{i+1}文目")
  print(temp3['input_ids'][i])
  print(temp3['attention_mask'][i])
  print(f"トークン長:{len(temp3['input_ids'][i])}")

4-10. return_token_type_ids：Falseにするとtoken_type_idが出力されない.  
return_tensors：出力する配列データの型.    
tf：tensorflow.    
pt：pytorch.     
np：numpy.

In [ ]:
temp3 = tokenizer2.batch_encode_plus(text_temp,
                                       max_length=14,
                                       padding='max_length',
                                       truncation=True,
                                       return_token_type_ids=False,
                                       return_tensors='np'
                                       )
temp3

4-11. 確認(それぞれ抽出)

In [ ]:
print(temp3['input_ids'])
print(temp3['attention_mask'])

#  扱うデータを読み込む.  
今回はChatGPTに作らせた架空のデータを用意.  
がんを疑う所見か否か

4-12. マウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

4-13. ファイルの読み込み

In [ ]:
import pandas as pd
# CSVファイルの読み込み
df = pd.read_csv('/content/drive/MyDrive/shuffled_medical_texts_labels.csv')
df

4-14.データを訓練用とテスト用に分割

In [ ]:
from sklearn.model_selection import train_test_split

# テキストとラベルを分離
texts = df['texts'].tolist()
labels = df['labels'].tolist()

# データを訓練用とテスト用に分割
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

中身の確認

In [ ]:
print(len(train_texts))
print(len(test_texts))
print(len(train_labels))
print(len(test_labels))

4-15.学習用と検証用をエンコード

In [ ]:
# エンコード
train_encodings = tokenizer2.batch_encode_plus(train_texts,
                                             max_length=512,
                                             padding='max_length',
                                              truncation=True,
                                              return_token_type_ids=False,
                                              return_tensors='tf')
train_encodings

In [ ]:
# エンコード
test_encodings = tokenizer2.batch_encode_plus(test_texts,
                                             max_length=512,
                                             padding='max_length',
                                              truncation=True,
                                              return_token_type_ids=False,
                                              return_tensors='tf')
test_encodings

# モデルの作成

4-16. モデルの作成

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from transformers import TFBertModel

# モデルの初期化
bert_model = TFBertModel.from_pretrained('cl-tohoku/bert-base-japanese-v3')

# BERTの入力
input_ids = Input(shape=(512,), dtype=tf.int32, name='input_ids')
attention_mask = Input(shape=(512,), dtype=tf.int32, name='attention_mask')

# BERTの出力
bert_output = bert_model(input_ids, attention_mask=attention_mask)[0]

# 分類のための層を追加
clf_output = bert_output[:, 0, :]
out = Dense(1, activation='sigmoid')(clf_output)

# モデルの定義
model2 = Model(inputs=[input_ids, attention_mask], outputs=out)

# モデルのコンパイル
model2.compile(optimizer=Adam(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model2.summary()

4-17.テストデータの中身の確認

In [ ]:
# zip(リスト1,リスト2)とすることで同時にfor文で処理できる
for i,j in zip(test_texts,test_labels):
  print(i,j)

4-18.学習前後で結果を比較するための学習前の予測

In [ ]:
# モデルに入力し予測
predictions_pre = model2.predict([test_encodings['input_ids'], test_encodings['attention_mask']])

# 予測結果の表示
print(predictions_pre)

4-19. 予測結果の比較(学習前)

In [ ]:
for i,j,k in zip(test_texts,test_labels,predictions_pre):
  print(i,"正解",j,"予測結果",k)

# 学習

4-20. 学習の実施

In [ ]:
# 学習
results = model2.fit({'input_ids': train_encodings['input_ids'],
           'attention_mask': train_encodings['attention_mask']},
          tf.convert_to_tensor(train_labels),
          validation_data=({'input_ids': test_encodings['input_ids'][2:],
                            'attention_mask': test_encodings['attention_mask'][2:]},
                           tf.convert_to_tensor(test_labels[2:])),
          epochs=3,
          batch_size=8)

4-21. 学習後の予測結果

In [ ]:
# モデルの予測
predictions = model2.predict([test_encodings['input_ids'], test_encodings['attention_mask']])
print(predictions)

4-21. 予測結果の比較(学習前後)

In [ ]:
# 学習前後の正解,学習前、学習後
for i,j,k,l in zip(test_texts,test_labels,predictions_pre,predictions):
  print(i,j,"学習前の予測",k,"→","学習後の予測",l)

4-22. 予測精度の評価

In [ ]:
# 評価(おまけ)

import numpy as np
from sklearn.metrics import accuracy_score, recall_score
from sklearn.metrics import precision_score, f1_score

# モデルの予測
predictions_round = np.round(predictions).flatten()  # 確率を0または1に丸める
print(f"19件の予測結果:{predictions_round}")
# 精度とリコールの計算
accuracy = accuracy_score(test_labels, predictions_round)
recall = recall_score(test_labels, predictions_round)

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")

# 精度とF1スコアの計算
precision = precision_score(test_labels, predictions_round)
f1 = f1_score(test_labels, predictions_round)

print(f"Precision: {precision}")
print(f"F1 Score: {f1}")